# Classification system

This notebook allows to do predictions using all the trained models (Virulence, Promiscuity and Resistance). The input needed is a list of sequences to be analyzed, and the output will be the predicted class for each of the sequences. The output will be both printed in the notebook itself and downloaded in a csv file.

## **Import Necessary Modules**

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
pip install biopython

     |████████████████████████████████| 2.3 MB 5.4 MB/s 


In [ ]:
import keras
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import np_utils
from random import shuffle
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Embedding, LSTM
from keras import regularizers, layers, preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from Bio.Seq import Seq
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

## **Model**

The following cells predict the class for each of the input sequences. Some results are shown here, although further details about them can be found in the 'Result' section of this notebook.

In [ ]:
Dataframe=pd.read_csv('ADD THE PATH FOR THE CSV WITH THE SEQUENCES TO BE STUDIED') # Add your here for the input sequences

In [ ]:
#Process of conversion from aminoacid sequence to codon sequence
Datacodons = Dataframe.copy()

Codons = list(Datacodons['genes'])


length = []
for n in range(len(Codons)):
    Codons[n] = list([Codons[n][i:i+3] for i in range(0, len(Codons[n]), 3)])
    length.append(len(Codons[n]))
    
Datacodons['codons'] = Codons

#The maximum length will be used for the padding of sequences
maxlen = max(length) # cut off after this number of codons in a list

max_words = 64 # Number of words in the dictionary, equal to the number of  possible codons
max_features = max_words


tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(DataCod['codons']))
sequences = tokenizer.texts_to_sequences(list(DataCod['codons']))
word_index = tokenizer.word_index
Xpad = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post', value=0)

gene_decoded=[]
for i in Xpad:
  gene=''
  for k in i:
    if k !=0:
      gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
  gene_decoded.append(gene)

# This step ensures that the generated dictionary has exactly what we asked for since in some cases words that are not codons are generated
new_dict= {'cac', 'cgt', 'aat', 'atg', 'tac', 'att', 'tgg', 'gac', 'tgc', 'act', 'gtt', 'gaa', 'aaa', 'cag', 'tga', 'ttg', 'gct', 'ttc', 'tct', 'ggc', 'aca', 'taa', 'ctg', 'ata', 'caa', 'ctt', 'tcg', 'gtc', 'aac', 'gga', 'acg', 'gca', 'tta', 'cta', 'acc', 'gat', 'tca', 'tat', 'agg', 'tgt', 'gtg', 'cga', 'cgc', 'ttt', 'aga', 'ggt', 'ctc', 'cca', 'ccg', 'gcg', 'tag', 'atc', 'cat', 'agt', 'cgg', 'aag', 'gag', 'cct', 'gta', 'ggg', 'tcc', 'agc', 'ccc', 'gcc'}
unwanted= set(word_index) - set(new_dict)
for unwanted_key in unwanted: del word_index[unwanted_key]

In [ ]:
main_model = keras.models.load_model('ADD THE PATH HERE FOR THE TRAINED MAIN DISCRIMINATOR MODEL') # Add here the path for the main discriminator model. We provided a trained model in the repository.

prediction = main_model.predict(Xpad)   # Prediction goes as [None,Promiscuity,Resistance,Virulence]

Preds = prediction.copy()
# Important! Transform the probabilities to binary decisions (either you belong to a label or not), we used the typical treshold in literature of 0.5
Preds[ np.where( Preds >= 0.5 ) ] = 1
Preds[ np.where( Preds < 0.5 ) ] = 0


# We'll store in different lists the sequences depending on the prediction given
# This will be the input of the following models, which will give a further 
# description of the sequences.
resistance = []
virulence = []
promiscuity = []
nodanger = []
for i in range(len(Preds)):
  if Preds[i][2] == 1:
    virulence.append(Xpad[i])
  if Preds[i][1] == 1:
    resistance.append(Xpad[i])
  if Preds[i][0] == 1:
    promiscuity.append(Xpad[i])
  if Preds[i][0]== 0 and Preds[i][1]== 0 and Preds[i][2]== 0:
    nodanger.append(Xpad[i])

print('Number of resistance sequences found: ' + str(len(resistance)))
print('Number of virulence sequences found: ' + str(len(virulence)))
print('Number of promiscuity sequences found: ' + str(len(promiscuity)))
print('Number of missclass sequences found: ' + str(len(nodanger)))

In [ ]:
# We define the dictionary describing each of the antibiotic resistance mechanisms
dd = {
    'Aminoglycosides': ['phosphorylation', 'acetylation', 'nucleotidylation', 'efflux', 'altered target'],
    'betalactams': ['hydrolysis', 'efflux', 'altered target'],
    'Glycopeptides': ['reprogramming peptidoglycan biosynthesis'],
    'Rifampin': ['ADP-ribosylation', 'efflux', 'altered target'],
    'MLS': ['hydrolysis', 'phosphorylation', 'nucleotidylation','acetylation','efflux', 'altered target'],
    'Tetracyclines': ['monooxygenation', 'efflux', 'altered target'],
    'Sulfonamides': ['efflux','altered target'],
    'Lipopeptides' : ['altered target'],
    'Phenicol' : ['acetylation', 'efflux', 'altered target'],
}
dd_s=dd.copy()
for key in dd_s:
  dd_s[key].sort()

# Here we will only predict the sequences that are classified as 'Resistance' in the main discriminator.

if len(resistance)!=0:
  # We modify the padding to 1395, which is the maximum length for the sequences used to train the model
  resistance_pad = pad_sequences(np.asarray(resistance), maxlen=1395, padding='post', truncating='post', value=0)
  
  resistant_model = keras.models.load_model('ADD THE PATH HERE FOR THE TRAINED RESISTANCE MODEL') # Add here the path for the resistance model. We provided a trained model in the repository.
  
  decoded=[]
  antibiotic=[]
  ambiguo=[]
  prediction = resistant_model.predict(resistance_pad)  
  Preds = prediction.copy()
  Preds[ np.where( Preds >= 0.5 ) ] = 1
  Preds[ np.where( Preds < 0.5 ) ] = 0
  for i in range(len(Preds)):
    vector=[]
    if Preds[i][0]==1:
      vector.append('ADP-ribosylation')
    if Preds[i][1]==1:
      vector.append('acetylation')
    if Preds[i][2]==1:
      vector.append('altered target')
    if Preds[i][3]==1:
      vector.append('efflux')
    if Preds[i][4]==1:
      vector.append('hydrolysis')
    if Preds[i][5]==1:
      vector.append('monooxygenation')
    if Preds[i][6]==1:
      vector.append('nucleotidylation')
    if Preds[i][7]==1:
      vector.append('phosphorylation')
    if Preds[i][8]==1:
      vector.append('reprogramming peptidoglycan biosynthesis')
    decoded.append(vector)

  for vec in decoded:
    try:
      antibiotic.append(list(dd_s.keys())[list(dd_s.values()).index(list(vec))])
    except ValueError:
      ambiguo.append(vec)

  print('\n----------------------------------------------------------------')
  print('Results ready in "Results" tab')

In [ ]:
  main={}
  for i in range(len(antibiotic)):
    unique, counts = np.unique(antibiotic[i], return_counts=True)
    dic=dict(zip(unique, counts))
    for key in dic:
      if key not in main:
        main[key]=[resistance[i]]
      else:
        main[key].append(resistance[i])
print(main.keys())

In [ ]:
# Here we will only predict the sequences that are classified as 'Virulence' in the main discriminator.
if len(virulence)!=0:
  virulence_model = keras.models.load_model('ADD THE PATH HERE FOR THE TRAINED VIRULENCE MODEL') # Add here the path for the virulence model. We provided a trained model in the repository.

  prediction = virulence_model.predict(np.asarray(virulence))   # [Adhesion, Toxin]
  Preds = prediction.copy()
  Preds[ np.where( Preds >= 0.5 ) ] = 1 
  Preds[ np.where( Preds < 0.5 ) ] = 0

  adhesion=[]
  Toxin=[]
  for i in range(len(Preds)):
    if Preds[i][0]==1:
      adhesion.append(virulence[i])
    if Preds[i][1]==1:
      Toxin.append(virulence[i])
  print('\n----------------------------------------------------------------')
  print('Number of adhesion mechanism sequences found: ' + str(len(adhesion)))
  print('Number of toxin mechanism sequences found: ' + str(len(Toxin)))

In [ ]:
# Here we will only predict the sequences that are classified as 'Virulence' in the main discriminator.
if len(promiscuity)!=0:
  promiscuity_pad = pad_sequences(np.asarray(promiscuity), maxlen=3066, padding='post', truncating='post', value=0)
  promiscuity_model = keras.models.load_model('ADD THE PATH HERE FOR THE TRAINED PROMISCUITY MODEL') # Add here the path for the promiscuity model. We provided a trained model in the repository.

  prediction = promiscuity_model.predict(promiscuity_pad)   
  Preds = prediction.copy()
  Preds[ np.where( Preds >= 0.5 ) ] = 1
  Preds[ np.where( Preds < 0.5 ) ] = 0

  transformation=[]
  conjugation=[]

  for i in range(len(Preds)):
    if Preds[i][0]==1:
      conjugation.append(promiscuity[i])
    if Preds[i][1]==1:
      transformation.append(promiscuity[i])

  print('\n----------------------------------------------------------------')
  print('Number of conjugation mechanism sequences found: ' + str(len(conjugation)))
  print('Number of transformation mechanism sequences found: ' + str(len(transformation)))


## **Results**

The results obtained for the input sequences are shown here. On the one hand, information about each of the sequences classified in each class is displayed in the notebook. Then, two different CSV files are generated. The first is organized into two columns: one for a RGB sequences asigned depending on the prediction (which will be used in other steps of the process) and the second one is the sequence itself. We call this CSV as the 'Colour CSV'. Finally, the second CSV contains a column with the predicted class and one with each sequence. We call this CSV as the 'Label CSV'.

In [ ]:
print('-------------------NUMBER OF RESISTANCE SEQUENCES-------------------')
print('nº: '+str(len(resistance)))

if len(resistance)!=0:
  ct=0
  Aminoglycosides=[]
  betalactams=[]
  Glycopeptides=[]
  Rifampin=[]
  MLS =[]
  Tetracyclines=[]
  Sulfonamides=[]
  Lipopeptides=[]
  Phenicol=[]
  for i in main:
    print('--- '+str(i)+' ---')
    print('nº: ',len(main[i]))
    print('-----------------------------' +'\n')
    for j in main[i]:
      ct+=1
      gene=''
      for k in j:
        if k !=0:
          gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
      print('gene nº '+str(ct) +': '+gene +'\n')
      if i=='Aminoglycosides':
        Aminoglycosides.append(gene)
      elif i=='betalactams':
        betalactams.append(gene)
      elif i=='Glycopeptides':
        Glycopeptides.append(gene)
      elif i=='Rifampin':
        Rifampin.append(gene)
      elif i=='MLS':
        MLS.append(gene)
      elif i=='Tetracyclines':
        Tetracyclines.append(gene)
      elif i=='Sulfonamides':
        Sulfonamides.append(gene)
      elif i=='Lipopeptides':
        Lipopeptides.append(gene)
      elif i=='Phenicol':
        Phenicol.append(gene)
    print('\n')
print('-------------------NUMBER OF VIRULENCE SEQUENCES-------------------')
print('nº: '+str(len(virulence)))

if len(virulence)!=0:
  ct=0
  print('--- Toxins ---')
  print('nº: ',len(Toxin))
  print('-----------------------------' +'\n')
  toxin_decoded=[]
  for i in Toxin:
    ct+=1
    gene=''
    for k in i:
      if k !=0:
        gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
    toxin_decoded.append(gene)
    print('gene nº '+str(ct) +': '+gene +'\n')
  print('\n')
  print('--- Adhesion ---')
  print('nº: ',len(adhesion))
  print('-----------------------------' +'\n')
  ct=0
  adhesion_decoded=[]
  for i in adhesion:
    ct+=1
    gene=''
    for k in i:
      if k !=0:
        gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
    adhesion_decoded.append(gene)
    print('gene nº '+str(ct) +': '+gene +'\n')

print('-------------------NUMBER OF PROMISCUITY SEQUENCES-------------------')
print('nº: '+str(len(promiscuity)))

if len(promiscuity)!=0:
  ct=0
  print('--- Conjugation ---')
  print('nº: ',len(conjugation))
  print('-----------------------------' +'\n')
  conjugation_decoded=[]
  for i in conjugation:
    ct+=1
    gene=''
    for k in i:
      if k !=0:
        gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
    conjugation_decoded.append(gene)
    print('gene nº '+str(ct) +': '+gene +'\n')
  print('\n')
  print('--- Transformation ---')
  print('nº: ',len(transformation))
  print('-----------------------------' +'\n')
  ct=0
  transformation_decoded=[]
  for i in transformation:
    ct+=1
    gene=''
    for k in i:
      if k !=0:
        gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
    transformation_decoded.append(gene)
    print('gene nº '+str(ct) +': '+gene +'\n')
    
if nodanger!=0:
  nodanger_decoded=[]
  for i in nodanger:
    gene=''
    for k in i:
      if k !=0:
        gene=gene+(list(word_index.keys())[list(word_index.values()).index(k)])
    nodanger_decoded.append(gene)

In [ ]:
results = open('ADD HERE THE PATH FOR WRITING THE COLOUR CSV','w')
results.write('Prediction,Genes''\n')
results2 = open('ADD HERE THE PATH FOR WRITING THE LABEL CSV','w')
results2.write('Prediction,Genes''\n')

c,f=2,len(gene_decoded)
Matrix=[[0 for i in range(c)] for j in range(f)]
Matrix2=[[0 for i in range(c)] for j in range(f)]
for i in range(len(gene_decoded)):
  Matrix[i][1]=gene_decoded[i]
  if gene_decoded[i] in Aminoglycosides:
    Matrix[i][0]='[128.128.128]'
    Matrix2[i][0]='Aminoglycosides'
  elif gene_decoded[i] in betalactams:
    Matrix[i][0]='[255.255.0]'
    Matrix2[i][0]='betalactams'
  elif gene_decoded[i] in Glycopeptides:
    Matrix[i][0]='[128.255.0]'
    Matrix2[i][0]='Glycopeptides'
  elif gene_decoded[i] in Rifampin:
    Matrix[i][0]='[0.255.0]'
    Matrix2[i][0]='Rifampin'
  elif gene_decoded[i] in MLS:
    Matrix[i][0]='[0.255.128]'
    Matrix2[i][0]='MLS'
  elif gene_decoded[i] in Tetracyclines:
    Matrix[i][0]='[0.255.255]'
    Matrix2[i][0]='Tetracyclines'
  elif gene_decoded[i] in Sulfonamides:
    Matrix[i][0]='[0.128.255]'
    Matrix2[i][0]='Sulfonamides'
  elif gene_decoded[i] in Lipopeptides:
    Matrix[i][0]='[0.0.255]'
    Matrix2[i][0]='Lipopeptides'
  elif gene_decoded[i] in Phenicol:
    Matrix[i][0]='[127.0.255]'
    Matrix2[i][0]='Phenicol'
  elif gene_decoded[i] in toxin_decoded:
    Matrix[i][0]='[255.0.255]'
    Matrix2[i][0]='Toxin'
  elif gene_decoded[i] in adhesion_decoded:
    Matrix[i][0]='[255.0.127]'
    Matrix2[i][0]='Adhesion'
  elif gene_decoded[i] in conjugation_decoded:
    Matrix[i][0]='[255.0.0]'
    Matrix2[i][0]='Conjugation'
  elif gene_decoded[i] in transformation_decoded:
    Matrix[i][0]='[255.128.0]'
    Matrix2[i][0]='Transformation'
  else:
    Matrix[i][0]='[0.0.0]'
    Matrix2[i][0]='Misclassification'


for i in Matrix:
  results.write(str(i[0])+','+str(i[1])+'\n')
results.close()
for i in Matrix2:
  results2.write(str(i[0])+','+str(i[1])+'\n')
results2.close()